In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('TC_AC_MNTHNG_CREDT_EXAMIN_20201204155835.csv')

In [4]:
data_copy = data.copy()

In [33]:
data_copy.drop(['시도명', '시군구명', '행정동명', '기업번호', '보증번호'], axis = 1, inplace = True)

KeyError: "['시도명' '시군구명' '행정동명' '기업번호' '보증번호'] not found in axis"

In [6]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37972 entries, 0 to 37971
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   기준년월      37972 non-null  object 
 1   성별코드      37972 non-null  object 
 2   연령대코드     37972 non-null  int64  
 3   사업자등록번호   37972 non-null  object 
 4   조사종류명     37972 non-null  object 
 5   부실구분명     37972 non-null  object 
 6   대위변제구분명   37972 non-null  object 
 7   회계년도      37972 non-null  int64  
 8   자기자본금액    37972 non-null  int64  
 9   당기순이익금액   37920 non-null  float64
 10  당기매출금액    37972 non-null  int64  
 11  총차입금액     37972 non-null  int64  
 12  운영자금차입금액  36774 non-null  float64
 13  부채비율      37972 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 4.1+ MB


In [7]:
# index 변경
data_copy.set_index('사업자등록번호', inplace = True)

In [8]:
# 결측치 제거
data_copy.dropna(inplace = True)

In [9]:
# 기준년월 datetime으로 변환
import datetime
data_copy['기준년월'] = pd.to_datetime(data_copy['기준년월'], format = "%Y-%m")

In [12]:
# 모듈 import
from sklearn.preprocessing import StandardScaler # 얘도 선형방식을 사용해서 전처리해줘야 성능 향상
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split

In [13]:
# 원핫 인코딩
ohe = OneHotEncoder()
df = data_copy[['성별코드', '연령대코드', '조사종류명', '대위변제구분명']]
df_ohe = ohe.fit_transform(df)
df_ohe = df_ohe.toarray()

In [14]:
cols = ohe.get_feature_names()

In [15]:
df_ohe = pd.DataFrame(df_ohe, columns = cols, index = df.index)
df_ohe

,x0_F,x0_M,x1_20,x1_30,x1_40,x1_50,x1_60,x1_70,x1_80,x2_소액심사,x2_정식심사,x3_대위변제,x3_대위변제해제
사업자등록번호,,,,,,,,,,,,,
11917*****,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
14181*****,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
12481*****,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
12681*****,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
13281*****,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14102*****,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
20481*****,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
12381*****,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [16]:
df_ohe = pd.DataFrame(df_ohe, columns = cols, index = df.index)

In [17]:
result = pd.concat([data_copy, df_ohe], axis = 1)
result

,기준년월,성별코드,연령대코드,조사종류명,부실구분명,대위변제구분명,회계년도,자기자본금액,당기순이익금액,당기매출금액,...,x1_30,x1_40,x1_50,x1_60,x1_70,x1_80,x2_소액심사,x2_정식심사,x3_대위변제,x3_대위변제해제
사업자등록번호,,,,,,,,,,,,,,,,,,,,,
11917*****,2020-10-01,F,60,정식심사,정상해지,대위변제해제,2009,293000000,62000000.0,1287000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
14181*****,2020-10-01,M,30,정식심사,정상해지,대위변제해제,2015,466000000,446000000.0,3135000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
12481*****,2020-10-01,M,60,정식심사,정상해지,대위변제해제,2008,623000000,231000000.0,2364000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
12681*****,2020-10-01,M,50,정식심사,정상해지,대위변제해제,2010,600000000,264000000.0,6972000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
13281*****,2020-10-01,M,60,정식심사,전액회수,대위변제해제,2005,0,16000000.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14102*****,2020-10-01,M,50,정식심사,정상해지,대위변제해제,2011,42000000,32000000.0,816000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
20481*****,2020-10-01,M,50,정식심사,정상,대위변제해제,2017,624000000,54000000.0,2605000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
12381*****,2020-10-01,M,60,정식심사,특수채권,대위변제,2005,0,11000000.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [18]:
# '부실구분명' => '정상' / '부실'로 구분
data['부실구분명'].value_counts()

정상해지    16641
정상      12588
특수채권     3673
구상채권     2633
전액회수     1628
관리종결      497
보증부실      192
매각        120
Name: 부실구분명, dtype: int64

In [47]:
df2 = pd.read_csv('신용평가_데이터1.csv')
df2

,사업자등록번호,기준년월,부실구분명,회계년도,자기자본금액,당기순이익금액,당기매출금액,총차입금액,운영자금차입금액,부채비율,...,x1_30,x1_40,x1_50,x1_60,x1_70,x1_80,x2_소액심사,x2_정식심사,x3_대위변제,x3_대위변제해제
0,11917*****,2020-10-01,정상해지,2009,293000000,62000000.0,1287000000,1775000000,320.0,332.08,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,14181*****,2020-10-01,정상해지,2015,466000000,446000000.0,3135000000,1590000000,160.0,154.29,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,12481*****,2020-10-01,정상해지,2008,623000000,231000000.0,2364000000,610000000,610.0,47.56,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,12681*****,2020-10-01,정상해지,2010,600000000,264000000.0,6972000000,2150000000,2150.0,169.67,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,13281*****,2020-10-01,전액회수,2005,0,16000000.0,0,100000000,100.0,130.50,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36719,14102*****,2020-10-01,정상해지,2011,42000000,32000000.0,816000000,252000000,252.0,564.28,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
36720,20481*****,2020-10-01,정상,2017,624000000,54000000.0,2605000000,249000000,249.0,23.23,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
36721,12381*****,2020-10-01,특수채권,2005,0,11000000.0,0,441000000,441.0,187.25,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
36722,27988*****,2020-10-01,정상,2018,120000000,22000000.0,1220000000,232000000,232.0,163.73,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [48]:
result2 = df2['부실구분명'].replace('정상해지', '정상')

In [49]:
result3 = result2.replace(['특수채권', '구상채권','전액회수', '관리종결', '보증부실', '매각' ], '부실')

In [50]:
result3

0        정상
1        정상
2        정상
3        정상
4        부실
         ..
36719    정상
36720    정상
36721    부실
36722    정상
36723    정상
Name: 부실구분명, Length: 36724, dtype: object

In [52]:
df2['부실여부'] =result3
df2

,사업자등록번호,기준년월,부실구분명,회계년도,자기자본금액,당기순이익금액,당기매출금액,총차입금액,운영자금차입금액,부채비율,...,x1_40,x1_50,x1_60,x1_70,x1_80,x2_소액심사,x2_정식심사,x3_대위변제,x3_대위변제해제,부실여부
0,11917*****,2020-10-01,정상해지,2009,293000000,62000000.0,1287000000,1775000000,320.0,332.08,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
1,14181*****,2020-10-01,정상해지,2015,466000000,446000000.0,3135000000,1590000000,160.0,154.29,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
2,12481*****,2020-10-01,정상해지,2008,623000000,231000000.0,2364000000,610000000,610.0,47.56,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
3,12681*****,2020-10-01,정상해지,2010,600000000,264000000.0,6972000000,2150000000,2150.0,169.67,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
4,13281*****,2020-10-01,전액회수,2005,0,16000000.0,0,100000000,100.0,130.50,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,부실
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36719,14102*****,2020-10-01,정상해지,2011,42000000,32000000.0,816000000,252000000,252.0,564.28,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
36720,20481*****,2020-10-01,정상,2017,624000000,54000000.0,2605000000,249000000,249.0,23.23,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
36721,12381*****,2020-10-01,특수채권,2005,0,11000000.0,0,441000000,441.0,187.25,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,부실
36722,27988*****,2020-10-01,정상,2018,120000000,22000000.0,1220000000,232000000,232.0,163.73,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상


In [67]:
df2

,기준년월,회계년도,자기자본금액,당기순이익금액,당기매출금액,총차입금액,운영자금차입금액,부채비율,x0_F,x0_M,...,x1_40,x1_50,x1_60,x1_70,x1_80,x2_소액심사,x2_정식심사,x3_대위변제,x3_대위변제해제,부실여부
사업자등록번호,,,,,,,,,,,,,,,,,,,,,
11917*****,2020-10-01,2009,293000000,62000000.0,1287000000,1775000000,320.0,332.08,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
14181*****,2020-10-01,2015,466000000,446000000.0,3135000000,1590000000,160.0,154.29,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
12481*****,2020-10-01,2008,623000000,231000000.0,2364000000,610000000,610.0,47.56,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
12681*****,2020-10-01,2010,600000000,264000000.0,6972000000,2150000000,2150.0,169.67,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
13281*****,2020-10-01,2005,0,16000000.0,0,100000000,100.0,130.50,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,부실
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14102*****,2020-10-01,2011,42000000,32000000.0,816000000,252000000,252.0,564.28,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
20481*****,2020-10-01,2017,624000000,54000000.0,2605000000,249000000,249.0,23.23,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
12381*****,2020-10-01,2005,0,11000000.0,0,441000000,441.0,187.25,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,부실


In [70]:
df2.set_index('사업자등록번호', inplace = True)
df2['기준년월'] = pd.to_datetime(df2['기준년월'], format ='%Y-%m')

KeyError: "None of ['사업자등록번호'] are in the columns"

In [67]:
df2

,기준년월,회계년도,자기자본금액,당기순이익금액,당기매출금액,총차입금액,운영자금차입금액,부채비율,x0_F,x0_M,...,x1_40,x1_50,x1_60,x1_70,x1_80,x2_소액심사,x2_정식심사,x3_대위변제,x3_대위변제해제,부실여부
사업자등록번호,,,,,,,,,,,,,,,,,,,,,
11917*****,2020-10-01,2009,293000000,62000000.0,1287000000,1775000000,320.0,332.08,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
14181*****,2020-10-01,2015,466000000,446000000.0,3135000000,1590000000,160.0,154.29,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
12481*****,2020-10-01,2008,623000000,231000000.0,2364000000,610000000,610.0,47.56,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
12681*****,2020-10-01,2010,600000000,264000000.0,6972000000,2150000000,2150.0,169.67,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
13281*****,2020-10-01,2005,0,16000000.0,0,100000000,100.0,130.50,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,부실
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14102*****,2020-10-01,2011,42000000,32000000.0,816000000,252000000,252.0,564.28,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
20481*****,2020-10-01,2017,624000000,54000000.0,2605000000,249000000,249.0,23.23,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
12381*****,2020-10-01,2005,0,11000000.0,0,441000000,441.0,187.25,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,부실


In [53]:
#기존부실구분명
df2.drop('부실구분명', axis = 1, inplace = True)

In [65]:
df2

,기준년월,회계년도,자기자본금액,당기순이익금액,당기매출금액,총차입금액,운영자금차입금액,부채비율,x0_F,x0_M,...,x1_40,x1_50,x1_60,x1_70,x1_80,x2_소액심사,x2_정식심사,x3_대위변제,x3_대위변제해제,부실여부
사업자등록번호,,,,,,,,,,,,,,,,,,,,,
11917*****,2020-10-01,2009,293000000,62000000.0,1287000000,1775000000,320.0,332.08,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
14181*****,2020-10-01,2015,466000000,446000000.0,3135000000,1590000000,160.0,154.29,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
12481*****,2020-10-01,2008,623000000,231000000.0,2364000000,610000000,610.0,47.56,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
12681*****,2020-10-01,2010,600000000,264000000.0,6972000000,2150000000,2150.0,169.67,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
13281*****,2020-10-01,2005,0,16000000.0,0,100000000,100.0,130.50,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,부실
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14102*****,2020-10-01,2011,42000000,32000000.0,816000000,252000000,252.0,564.28,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
20481*****,2020-10-01,2017,624000000,54000000.0,2605000000,249000000,249.0,23.23,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,정상
12381*****,2020-10-01,2005,0,11000000.0,0,441000000,441.0,187.25,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,부실


In [76]:
df2.drop('기준년월',axis = 1, inplace = True)


KeyError: "['기준년월'] not found in axis"

In [77]:
df2.columns

Index(['회계년도', '자기자본금액', '당기순이익금액', '당기매출금액', '총차입금액', '운영자금차입금액', '부채비율',
       'x0_F', 'x0_M', 'x1_20', 'x1_30', 'x1_40', 'x1_50', 'x1_60', 'x1_70',
       'x1_80', 'x2_소액심사', 'x2_정식심사', 'x3_대위변제', 'x3_대위변제해제', '부실여부'],
      dtype='object')

In [99]:
# X, y 나누기
y = df2['부실여부']
X = df2.drop('부실여부', axis = 1)

In [92]:
# Train, Test로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2 ,random_state = 1)

# Train, Validation Dataset으로 분리
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size = 0.2,
                                                  stratify = y_train,
                                                  random_state = 1)
y_train.shape, y_val.shape, y_test.shape


((23503,), (5876,), (7345,))

In [93]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23503 entries, 13487***** to 12381*****
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   회계년도       23503 non-null  int64  
 1   자기자본금액     23503 non-null  int64  
 2   당기순이익금액    23503 non-null  float64
 3   당기매출금액     23503 non-null  int64  
 4   총차입금액      23503 non-null  int64  
 5   운영자금차입금액   23503 non-null  float64
 6   부채비율       23503 non-null  float64
 7   x0_F       23503 non-null  float64
 8   x0_M       23503 non-null  float64
 9   x1_20      23503 non-null  float64
 10  x1_30      23503 non-null  float64
 11  x1_40      23503 non-null  float64
 12  x1_50      23503 non-null  float64
 13  x1_60      23503 non-null  float64
 14  x1_70      23503 non-null  float64
 15  x1_80      23503 non-null  float64
 16  x2_소액심사    23503 non-null  float64
 17  x2_정식심사    23503 non-null  float64
 18  x3_대위변제    23503 non-null  float64
 19  x3_대위변제해제  23503 non-null  float64
dt

In [91]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36724 entries, 11917***** to 12708*****
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   회계년도       36724 non-null  int64  
 1   자기자본금액     36724 non-null  int64  
 2   당기순이익금액    36724 non-null  float64
 3   당기매출금액     36724 non-null  int64  
 4   총차입금액      36724 non-null  int64  
 5   운영자금차입금액   36724 non-null  float64
 6   부채비율       36724 non-null  float64
 7   x0_F       36724 non-null  float64
 8   x0_M       36724 non-null  float64
 9   x1_20      36724 non-null  float64
 10  x1_30      36724 non-null  float64
 11  x1_40      36724 non-null  float64
 12  x1_50      36724 non-null  float64
 13  x1_60      36724 non-null  float64
 14  x1_70      36724 non-null  float64
 15  x1_80      36724 non-null  float64
 16  x2_소액심사    36724 non-null  float64
 17  x2_정식심사    36724 non-null  float64
 18  x3_대위변제    36724 non-null  float64
 19  x3_대위변제해제  36724 non-null  float64
dt

In [100]:
y 

사업자등록번호
11917*****    정상
14181*****    정상
12481*****    정상
12681*****    정상
13281*****    부실
              ..
14102*****    정상
20481*****    정상
12381*****    부실
27988*****    정상
12708*****    정상
Name: 부실여부, Length: 36724, dtype: object

In [94]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

In [95]:
#모델 생성 및 학습 : 리니어리그레션 과 같은 모듈에 들어가잇음
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report #분류라서 정확도 보자

lr = LogisticRegression(random_state = 1, C=1, max_iter = 100) # 가중치를 랜덤으로 잡는데 그걸 조정하기 위해서 랜덤스테이트 지정
lr.fit(X_train_scaled, y_train) #가중치 찾는거 1000번까지 최대로 하고 그만해

LogisticRegression(C=1, random_state=1)

In [96]:
#예측
pred_train = lr.predict(X_train_scaled)
pred_val = lr.predict(X_val_scaled)

In [97]:
#검증
acc_train = accuracy_score(y_train, pred_train)
acc_val = accuracy_score(y_val, pred_val)
print(acc_train, acc_val)

0.9097562013359997 0.9075901974132062
